In [1]:
# Same as the ManyToOne example, except OneToOne is intended to model
# is-a (inheritance) relationships

In [2]:
# Clean up in case of re-running
OneToOneChildProtect.objects.all().delete()
OneToOneParent.objects.all().delete()
OneToOneChildCascade.objects.all().delete()
OneToOneChildNull.objects.all().delete();

In [3]:
# Populate a parent with all 3 kinds of children.
parent = OneToOneParent.objects.create()

[OneToOneChildCascade.objects.create(parent=parent) for _ in range(2)]
[OneToOneChildNull.objects.create(parent=parent) for _ in range(2)]
OneToOneChildProtect.objects.create(parent=parent)

[p.print_relations() for p in OneToOneParent.objects.all()];

OneToOneParent id: 1 has cascade children ['OneToOneChildCascade 2', 'OneToOneChildCascade 3']
OneToOneParent id: 1 has null children ['OneToOneChildNull 4', 'OneToOneChildNull 5']
OneToOneParent id: 1 has protect children ['OneToOneChildProtect 6']


In [4]:
# Deletion behavior of the child does not depend on the type.
parent.o_children_cascade.last().delete()
parent.o_children_null.last().delete()
parent.o_children_protect.last().delete()

[p.print_relations() for p in OneToOneParent.objects.all()];

Delete called on OneToOneChildCascade 3
Delete called on OneToOneChildNull 5
Delete called on OneToOneChildProtect 6
OneToOneParent id: 1 has cascade children ['OneToOneChildCascade 2']
OneToOneParent id: 1 has null children ['OneToOneChildNull 4']
OneToOneParent id: 1 has protect children []


In [5]:
# But when deleting the parent, the behavior depends on on_delete in the child ForeignKey to the parent
def get_ids_parents(ModelClass):
    return [t for t in ModelClass.objects.values_list("id", "parent", named=True)]

print('Existing children')
print(f'ManyToOneChildCascade: {get_ids_parents(OneToOneChildCascade)}')
print(f'ManyToOneChildNull: {get_ids_parents(OneToOneChildNull)}')

print('\nDeleting the parent of these children\n')
parent.delete()

# Here be the dragon!
print(f'ManyToOneChildCascade: {get_ids_parents(OneToOneChildCascade)} <--Got deleted with the parent!')
print(f'ManyToOneChildNull: {get_ids_parents(OneToOneChildNull)} <-- Orphan :_( ');

Existing children
ManyToOneChildCascade: [Row(id=2, parent=1)]
ManyToOneChildNull: [Row(id=4, parent=1)]

Deleting the parent of these children

Delete called on OneToOneParent 1
ManyToOneChildCascade: [] <--Got deleted with the parent!
ManyToOneChildNull: [Row(id=4, parent=None)] <-- Orphan :_( 


In [6]:
# Now the final case - Try to delete a parent protected by a child
from django.db.models import ProtectedError

parent = OneToOneParent.objects.create()
OneToOneChildProtect.objects.create(parent=parent)

try:
    parent.delete()
    print('\nThe parent was deleted!\n')
except ProtectedError:
    print('\nThe parent could not be deleted!\n')
    parent.print_relations()

Delete called on OneToOneParent 7

The parent could not be deleted!

OneToOneParent id: 7 has cascade children []
OneToOneParent id: 7 has null children []
OneToOneParent id: 7 has protect children ['OneToOneChildProtect 8']


In [7]:
# The child is protecting the parent as long as it exists.  One use of this is to enforce clean-up,
# there can be no orhpaned child with a protected parent.
parent.o_children_protect.first().delete()
parent.delete();

Delete called on OneToOneChildProtect 8
Delete called on OneToOneParent 7
